Requirements

In [91]:
!pip install sklearn
!pip install quandl
!pip install pandas
!pip install bs4
!pip install xgboost
!pip install sklearn
!pip install IPython
!pip install matplotlib
!pip install torch

print("Requirements Installed")

Requirements Installed


This parses the bracket using the Beautiful Soup package, and the resulting bracket is in the form of a list, named "results"

In [92]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def getBracket(year):
  year = str(year)
  source = requests.get('https://www.sports-reference.com/cbb/postseason/'+year+'-ncaa.html').text
  soup = BeautifulSoup(source, 'lxml')

  all_rounds = soup.find_all('div', class_ = 'round')

  results = []

  for round in all_rounds:
    teams_in_round = round.find_all('div')
    counter = 0

    for matchup in teams_in_round:
      if counter%3 == 0:
        teams = matchup.find_all('div')
        for team in teams:
          info = team.a.text + " " + team.span.text
          results.append(info)
      counter+=1

  return results


This parses the analytical statistics and inserts them into a Pandas Dataframe

In [93]:
def getStats(year):
  cols = ["Rank","Name+Seed","Conference","W-L","AdjEM","AdjO","R_AdjO","AdjD","R_AdjD","AdjT","R_AdjT","Luck","R_Luck","AdjEM1","R_AdjEM1","OppO","R_OppO","OppD","R_OppD","AdjEM2","R_AdjEM2"]
  df = pd.DataFrame([],columns = cols)

  year = str(year)
  source = requests.get('https://kenpom.com/index.php?y='+year).text
  soup = BeautifulSoup(source, 'lxml')


  all_teams = soup.find('tbody')
  all_teams = all_teams.find_all('tr')


  for team in all_teams:
    stats = team.find_all('td')
    temp_df = []
    for num in stats:
      temp_df.append(num.text)
    if len(temp_df) == 0:
      continue
    temp = pd.DataFrame([temp_df],columns = cols)
    df = df.append(temp,ignore_index=True)

  df['Win']=0

  return df

There are some inconsistencies with the naming of some teams, so this normalizes them.

In [94]:
#(bracket,kenpom -- correct)
def cleanNames(year):
  data = pd.DataFrame()
  df = getStats(year)
  for team in getBracket(year):
    if "State" in team:
      team = team.replace("State","St.")
    if "UConn" in team:
      team = team.replace("UConn","Connecticut")
    if "NC St." in team:
      team = team.replace("NC St.","N.C. State")
    if "St. John's (NY)" in team:
      team = team.replace("St. John's (NY)","St. John's")
    if "Pitt" in team:
      team = team.replace("Pitt","Pittsburgh")
    if "UIC" in team:
      team = team.replace("UIC","Illinois Chicago")
    if "UCSB" in team:
      team = team.replace("UCSB","UC Santa Barbara")
    if "Ole Miss" in team:
      team = team.replace("Ole Miss","Mississippi")
    if "Miami (FL)" in team:
      team = team.replace("Miami (FL)","Miami FL")
    if "ETSU" in team:
      team = team.replace("ETSU","East Tennessee St.")
    if "Troy" in team:
      if year != 2017:
        team = team.replace("Troy","Troy St.")
    if "St. Joseph's" in team:
      team = team.replace("St. Joseph's","Saint Joseph's")
    if "UNC" in team:
      if "Wilmington" not in team:
        if  "Asheville" not in team:
          if "Greensboro" not in team:
            team = team.replace("UNC","North Carolina")
    if "Louisiana" in team:
      if "South" not in team:
        team = team.replace("Louisiana","Louisiana Lafayette")
    if "Alabama-Birmingham" in team:
      team = team.replace("Alabama-Birmingham","UAB")
    if "Albany (NY)" in team:
      team = team.replace("Albany (NY)","Albany")
    if "Texas A&M-Corpus Christi" in team:
      team = team.replace("Texas A&M-Corpus Christi","Texas A&M Corpus Chris")
    if "Miami (OH)" in team:
      team = team.replace("Miami (OH)","Miami OH")
    if "Texas-Arlington" in team:
      team = team.replace("Texas-Arlington","UT Arlington")
    if "Arkansas-Pine Bluff" in team:
      team = team.replace("Arkansas-Pine Bluff","Arkansas Pine Bluff")
    if "LIU" in team:
      team = team.replace("LIU","LIU Brooklyn")
    if "St. Peter's" in team:
      team = team.replace("St. Peter's","Saint Peter's")
    if "Loyola (MD)" in team:
      team = team.replace("Loyola (MD)","Loyola MD")
    if "UMass" in team:
      team = team.replace("UMass","Massachusetts")
    if "UC-Irvine" in team:
      team = team.replace("UC-Irvine","UC Irvine")
    if "Little Rock" in team:
      team = team.replace("Little Rock","Arkansas Little Rock")
    if "UC-Davis" in team:
      team = team.replace("UC-Davis","UC Davis")
    if "Loyola (IL)" in team:
      team = team.replace("Loyola (IL)","Loyola Chicago")
    if "Gardner-Webb" in team:
      team = team.replace("Gardner-Webb","Gardner Webb")
    r = df.loc[(df["Name+Seed"] == team)]
    data = data.append(r)

  return data


In [95]:
#print(cleanNames(2019))


for year in range(2002,2020):
  if len(cleanNames(year)) !=  131:
    print("{} needs a fix by {}".format(year,131-len(cleanNames(year))))

Loops through the data structure and adds a parameter called "Win". This is the result of the game when the two adjacent teams play each other. For example, if the structure had the state [Team 1, Team 2, Team 3, Team 4], 

In [96]:

#print(data)

def get_winners(beginning,end,data):
  data = data.reset_index(drop=True)
  pd.set_option('display.max_rows',132)
  adder = 16
  for i in range(beginning,end,2):
    if (data.iloc[i]['Name+Seed'] == data.iloc[i+adder]['Name+Seed']):
      data.at[i,'Win'] = 1
    else:
      data.at[i+1,'Win'] = 1
    adder = adder - 1
  return data


def clean_winners(data):
  if (data.iloc[124]['Name+Seed'] == data.iloc[128]['Name+Seed']):
    data.at[124,'Win'] = 1
  else:
    data.at[125,'Win'] = 1

  if (data.iloc[126]['Name+Seed'] == data.iloc[129]['Name+Seed']):
    data.at[126,'Win'] = 1
  else:
    data.at[127,'Win'] = 1

  if (data.iloc[128]['Name+Seed'] == data.iloc[130]['Name+Seed']):
    data.at[128,'Win'] = 1
  else:
    data.at[129,'Win'] = 1

  dups = [30,61,92,123,130]
  for i in dups:
    data = data.drop([i])

  data = data.reset_index(drop=True)
  pd.set_option('display.max_rows',130)
  return data

Crates two new parameters; win percentage and the seed of the team

In [97]:


import sklearn
import numpy as np
import re
from sklearn import linear_model
from sklearn.utils import shuffle
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.svm import SVC
import functools


def normalize_data(data,teams):
  pd.set_option('display.max_columns',None)



  winpercentages = []
  for record in range(teams):
    record = data.at[record,"W-L"]
    seperate = record.split('-')
    wins = seperate[0]
    losses = seperate[1]
    wins,losses = float(wins),float(losses)
    pctg = wins/(wins+losses)
    winpercentages.append(pctg)


  seeds = []
  for seed in range(teams):
    seed = data.at[seed,"Name+Seed"]
    match = re.findall(r'\d+',seed,flags=0)
    seeds.append(match[0])

  copy_df = data


  data['WinPercent'] = winpercentages
  data['Seeds'] = seeds


  pd.set_option('display.max_rows',130)

  return data







Scales the data and converts to tensors to prepare for training

In [98]:
def scale_data(data):

  #removed = ["Name+Seed","Conference","W-L","AdjO","AdjD","AdjT","R_Luck","Luck","R_AdjEM1","R_AdjEM2","R_OppO","R_OppD","AdjEM","AdjEM1","OppO","OppD","AdjEM2"]
  removed = ["Name+Seed","Conference","W-L"]
  for i in removed:
    data = data.drop([i],1)
  predict = "Win"
  y = data[predict]
  y = y.astype('float32')
  data = data.drop([predict],1)

  cols = [data.columns.tolist()]
  for col in cols:
    data[col] = scale(data[col])
    data[col] = data[col].astype('float32')

  return data, y



Splits the tournament data from 2002-2019 into training and testing sets to select the best fitting model.

In [99]:
#epochs = 2
#print(y)
#getBracket(2002)
#getStats(2002)

years = [2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2018,2019]

final_data = pd.DataFrame()
final_y = pd.Series(dtype='object')

for year in years:
  data = cleanNames(year)
  data=get_winners(0,30,data)
  data=get_winners(31,61,data)
  data=get_winners(62,92,data)
  data=get_winners(93,123,data)
  data=clean_winners(data)
  data = normalize_data(data,126)
  data,y=scale_data(data)
  final_data = final_data.append(data)
  final_y = final_y.append(y)


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(final_data,final_y,train_size=0.8)

print("-----------------")
print("Logistic")
logistic = linear_model.LogisticRegression()
logistic.fit(x_train,y_train)
accuracy = logistic.score(x_train,y_train)
print("Training: ",accuracy)
accuracy = logistic.score(x_test,y_test)
print("Testing: ",accuracy)
#print("Co: \n",logistic.coef_)
#print("Intercept: \n", logistic.intercept_)

print("-----------------")

print("XGBoost")

x = xgb.XGBClassifier()
x.fit(x_train,y_train)
accuracy = x.score(x_train,y_train)
print("Training: ",accuracy)
accuracy = x.score(x_test,y_test)
print("Testing: ",accuracy)


print("-----------------")

print("Support Vector Machine")

vector = SVC()
vector.fit(x_train,y_train)
accuracy = vector.score(x_train,y_train)
print("Training: ",accuracy) 
accuracy = vector.score(x_test,y_test)
print("Testing: ",accuracy) 
print("\n \n \n") 

-----------------
Logistic
Training:  0.7098657326328079
Testing:  0.7249417249417249
-----------------
XGBoost
Training:  0.7419731465265615
Testing:  0.6806526806526807
-----------------
Support Vector Machine
Training:  0.7098657326328079
Testing:  0.6946386946386947

 
 



In [100]:
def my_main(year):

  data = cleanNames(year)
  data=get_winners(0,30,data)
  data=get_winners(31,61,data)
  data=get_winners(62,92,data)
  data=get_winners(93,123,data)
  data=clean_winners(data)
  data = normalize_data(data,126)
  data,y=scale_data(data)

  return data,y 


def no_clean(year):
  data = cleanNames(year)
  data=get_winners(0,30,data)
  data=get_winners(31,61,data)
  data=get_winners(62,92,data)
  data=get_winners(93,123,data)
  data=clean_winners(data)
  return data

# call no_clean first?
# restrucures data into bracket form
def restructure_data(year):
  data = no_clean(year)
  new = pd.DataFrame()
  new = data[0:16]
  new = new.append(data[30:46])
  new = new.append(data[60:76])
  new = new.append(data[90:106])

  new = new.append(data[16:24])
  new = new.append(data[46:54])
  new = new.append(data[76:84])
  new = new.append(data[106:116])

  new = new.append(data[24:28])
  new = new.append(data[54:58])
  new = new.append(data[84:88])
  new = new.append(data[114:118])

  new = new.append(data[28:30])
  new = new.append(data[58:60])
  new = new.append(data[88:90])
  new = new.append(data[118:120])

  new = new.append(data[120:])

  if (data.loc[124]['Win']>data.loc[125]['Win']):
    new = new.append(data.loc[124])
  else:
    new = new.append(data.loc[125])

  new = new.reset_index(drop=True)

  return new


year = 2017

pd.options.mode.chained_assignment = None 

pd.set_option('display.max_columns',6)
my_bracket = no_clean(year)
og_x,y = my_main(year)
x = logistic.predict_proba(og_x)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
my_bracket['Predicted Outcome'] = mine
og_x['Predicted Outcome'] = mine
####
drops = [(16,30),(46,60),(76,90),(106,126)]
for i in drops:
  for j in range(i[0],i[1]):
    my_bracket = my_bracket.drop([j])
    og_x = og_x.drop([j])
####


round_32 = my_bracket[my_bracket['Predicted Outcome']==1]
y = og_x[og_x['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)

round_32.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine


  

round_16 = round_32[round_32['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_16.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine




round_8 = round_16[round_16['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_8.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine






round_4 = round_8[round_8['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_4.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine




round_2 = round_4[round_4['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_2.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine

winner = round_2[round_2['Predicted Outcome']==1]



#print("Round of 64")
#print(my_bracket['Name+Seed'])

#print("Round of 32")
#print(round_32['Name+Seed'])

#print("Round of 16")
#print(round_16['Name+Seed'])

#print("Round of 8")
#print(round_8['Name+Seed'])

#print("Round of 4")
#print(round_4['Name+Seed'])

#print("Round of 2")
#print(round_2['Name+Seed'])


#print("Winner")
#print(winner['Name+Seed'])


#print(restructure_data(2015))

actual = restructure_data(year)

#print(actual[64:96]['Name+Seed'])
num_correct = len(set(round_32['Name+Seed']).intersection(set(actual[64:96]['Name+Seed'])))
print("32-team accuracy is {}".format(num_correct/32))

num_correct = len(set(round_16['Name+Seed']).intersection(set(actual[96:112]['Name+Seed'])))
print("16-team accuracy is {}".format(num_correct/16))

num_correct = len(set(round_8['Name+Seed']).intersection(set(actual[112:120]['Name+Seed'])))
print("8-team accuracy is {}".format(num_correct/8))

num_correct = len(set(round_4['Name+Seed']).intersection(set(actual[120:124]['Name+Seed'])))
print("4-team accuracy is {}".format(num_correct/4))

num_correct = len(set(round_2['Name+Seed']).intersection(set(actual[124:126]['Name+Seed'])))
print("2-team accuracy is {}".format(num_correct/2))

num_correct = len(set(winner['Name+Seed']).intersection(set(actual[126:127]['Name+Seed'])))
print("Winner accuracy is {}".format(num_correct/1))


32-team accuracy is 0.875
16-team accuracy is 0.625
8-team accuracy is 0.625
4-team accuracy is 0.5
2-team accuracy is 0.0
Winner accuracy is 0.0
